# Activities extraction

Questo notebook crea csv per ogni activity tramite chunks di testo.
Ogni csv è specifico di 1 attività.

In [13]:
import pandas as pd
import re
import ast

df_corpus = pd.read_csv('camera.csv')

df_corpus['camera'] = df_corpus['camera'].apply(ast.literal_eval)

df_corpus.head()

,id,camera
0,25G00016,[Camera dei deputati (atto n. 400): Presentato...
1,25G00014,[Camera dei deputati (atto n. 2206): Assegnato...
2,25G00011,[Camera dei deputati (atto n. 1835): Presentat...
3,25G00009,[Camera dei deputati (atto n. 2196): Assegnato...
4,25G00008,[Camera dei deputati (atto n. 2031): Assegnato...


In [14]:
print(type(df_corpus['camera']))

<class 'pandas.core.series.Series'>


In [ ]:
def find_presentato_da_and_date(df_corpus):
    results = []  # Lista per raccogliere i risultati
    
    # Regex per 'presentato da' e per una data
    regex_presentato = r"presentato da(?:ll'|l|gli|\s)+([^,\.]+)"
    regex_date = r"(\d{1,2})\s([a-zA-Zà-ùÀ-Ù]+)\s(\d{4})"
    
    for index, row in df_corpus.iterrows():
        sentences = row['camera']  # Lista di frasi già estratte
        #print(sentences)
        
        for sentence in sentences:
            # Cerca il match per 'presentato da' nella frase
            match_presentato = re.search(regex_presentato, sentence, re.IGNORECASE)
            
            if match_presentato:
                #print("Frase trovata:", sentence)  # Debug: mostra la frase trovata
                
                # Cerca la data nella stessa frase
                match_date = re.search(regex_date, sentence)
                
                # Aggiungi la tupla (id, presentato_da, data)
                results.append((
                    row['id'], 
                    match_presentato.group(0).strip(), 
                    match_date.group(0).strip() if match_date else None
                ))
                
                # Una volta trovato, possiamo interrompere la ricerca in altre frasi
                break
    
    return results

# Esegui la funzione
result = find_presentato_da_and_date(df_corpus)

# Per visualizzare il risultato
for r in result:
    print(r)

# Print tupla con (,None)
print ("\nTupla con data 'None':")
for r in result:
    if r[2] is None:
        print(r)
        print(df_corpus.query(f'id == "{r[0]}"')['camera'].iloc[0])

('25G00016', "Presentato dall'Onorevole Marco Simiani (PD-IDP) e altri", '19 ottobre 2022')
('25G00011', "Presentato dall'Onorevole Giorgio Mulè (FI-PPE) e altri", '22 aprile 2024')
('24G00219', 'Presentato da Onorevole Massimiliano Panizzut (Lega)', '13 ottobre 2022')
('24G00229', "presentato dal Ministro dell'economia e finanze Giancarlo Giorgetti (Governo Meloni-I)", '29 ottobre 2024')
('24G00218', 'Presentato dal Ministro del lavoro e politiche sociali Marina Elvira Calderone (Governo Meloni-I)', None)
('24G00217', 'Presentato dal Presidente del Consiglio dei ministri', '28 ottobre 2024')
('24G00215', 'Presentato dal Ministro delle imprese e del made in Italy Adolfo Urso e dal Ministro delle infrastrutture e dei trasporti Matteo Salvini (Governo Meloni-I)', '9 agosto 2024')
('24G00207', "Presentato dall'Onorevole Toni Ricciardi (PD-IDP)", '7 marzo 2023')
('24G00206', 'Presentato dal Presidente del Consiglio Giorgia Meloni', '11 ottobre 2024')
('24G00199', 'Presentato dal Ministro d

In [ ]:
# Crea DataFrame
df_event_log_presentazione = pd.DataFrame(result, columns=['case_id', 'chunk', 'time'])
df_event_log_presentazione['activity'] = 'presentazione'
df_event_log_presentazione['chamber'] = 'camera'

# Salva come CSV
df_event_log_presentazione.to_csv('event_log_presentazione.csv', index=False)

# Verifica il risultato
print(df_event_log_presentazione.head())

     case_id                                              chunk  \
0   25G00016  Presentato dall'Onorevole Marco Simiani (PD-ID...   
1   25G00011  Presentato dall'Onorevole Giorgio Mulè (FI-PPE...   
2   24G00219  Presentato da Onorevole Massimiliano Panizzut ...   
3   24G00229  presentato dal Ministro dell'economia e finanz...   
4   24G00218  Presentato dal Ministro del lavoro e politiche...   
5   24G00217  Presentato dal Presidente del Consiglio dei mi...   
6   24G00215  Presentato dal Ministro delle imprese e del ma...   
7   24G00207  Presentato dall'Onorevole Toni Ricciardi (PD-IDP)   
8   24G00206  Presentato dal Presidente del Consiglio Giorgi...   
9   24G00199  Presentato dal Ministro delle Infrastrutture e...   
10  24G00194  Presentato dall'Onorevole Marco Furfaro (PD) e...   
11  24G00186  Presentato dall'Onorevole Alessandro Battilocc...   
12  24G00187  Presentato dall'Onorevole Maria Carolina Varch...   
13  24G00184  Presentato dal Presidente del Consiglio dei mi..

In [46]:
def find_assegnato_a_and_date(df_corpus):
    results = []  # Lista per raccogliere i risultati
    
    # Regex per 'assegnato a' e per una data
    #regex_presentato = r"assegnato a(?:lla|lle|\s)+(\w|\s)+parer(?:e|i)"
    regex_presentato = r"assegnato a(?:lla|lle)[\s|\w|\W]*$"
    regex_date = r"(\d{1,2})\s([a-zA-Zà-ùÀ-Ù]+)\s(\d{4})"
    
    for index, row in df_corpus.iterrows():
        sentences = row['camera']  # Lista di frasi già estratte
        
        for sentence in sentences:
            # Cerca il match per 'presentato da' nella frase
            match_presentato = re.search(regex_presentato, sentence, re.IGNORECASE)
            
            if match_presentato:
                print("Frase trovata:", sentence)  # Debug: mostra la frase trovata
                
                # Cerca la data nella stessa frase
                match_date = re.search(regex_date, sentence)
                
                # Aggiungi la tupla (id, presentato_da, data)
                results.append((
                    row['id'], 
                    #match_presentato.group(0).strip(), 
                    sentence,
                    match_date.group(0).strip() if match_date else None
                ))
                
                # Una volta trovato, possiamo interrompere la ricerca in altre frasi
                break
    
    return results

# Esegui la funzione
result = find_assegnato_a_and_date(df_corpus)

# Per visualizzare il risultato
for r in result:
    print(r)

# Print tupla con (,None)
for r in result:
    if r[2] is None:
        print(r)
        print(df_corpus.query(f'id == "{r[0]}"')['camera'].iloc[0])

Frase trovata: Assegnato alla VIII Commissione (Ambiente, territorio e lavori pubblici), in sede referente, il 23 novembre 2022, con i pareri delle commissioni I (Affari costituzionali, della Presidenza del Consiglio e Interni), II (Giustizia), V (Bilancio, tesoro e programmazione), XI (Lavoro pubblico e privato) e per le Questioni regionali.
Frase trovata: Camera dei deputati (atto n. 2206): Assegnato alle Commissioni riunite III (Affari esteri e comunitari) e IV (Difesa), in sede referente, il 22 gennaio 2025, con i pareri delle Commissioni I (Affari costituzionali, della Presidenza del Consiglio e Interni), V (Bilancio, tesoro e programmazione) e XIV (Politiche dell'Unione europea).
Frase trovata: Assegnato alla IV Commissione (Difesa), in sede referente, il 3 giugno 2024, con i pareri delle commissioni I (Affari costituzionali, della Presidenza del Consiglio e interni), V (Bilancio, Tesoro e Programmazione), VII (Cultura, scienza e istruzione), e per le Questioni regionali.
Frase t

In [47]:
# Crea DataFrame
df_event_log_assegnazione = pd.DataFrame(result, columns=['id', 'chunk', 'time'])
df_event_log_assegnazione['activity'] = 'assegnazione'
df_event_log_assegnazione['chamber'] = 'camera'

# Salva come CSV
df_event_log_assegnazione.to_csv('event_log_assegnazione.csv', index=False)

# Verifica il risultato
print(df_event_log_assegnazione.head())

         id                                              chunk  \
0  25G00016  Assegnato alla VIII Commissione (Ambiente, ter...   
1  25G00014  Camera dei deputati (atto n. 2206): Assegnato ...   
2  25G00011  Assegnato alla IV Commissione (Difesa), in sed...   
3  25G00009  Camera dei deputati (atto n. 2196): Assegnato ...   
4  25G00008  Camera dei deputati (atto n. 2031): Assegnato ...   

                time      activity chamber  
0   23 novembre 2022  assegnazione  camera  
1    22 gennaio 2025  assegnazione  camera  
2      3 giugno 2024  assegnazione  camera  
3    15 gennaio 2025  assegnazione  camera  
4  16 settembre 2024  assegnazione  camera  
